# Track filtering/fitting with LSTMs and Gaussian predictions

This is a continuous space model using the ACTS data.
Like the LSTMTrackFilter notebook, it predicts the location of the next hit given a hit sequence.
However, unlike the previous model, we now produce a probability distribution in the form of a multivariate Gaussian.

This lets our model quantify uncertainty and score hits in a more meaningful way.

In [1]:
# Select a GPU first
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'
cuda = False

In [2]:
from __future__ import print_function

# Data libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.stats import norm, multivariate_normal

# Torch imports
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

# Visualization
import matplotlib.pyplot as plt

# Local imports
from data import process_files
import torchutils
torchutils.set_cuda(cuda)
from torchutils import np_to_torch, torch_zeros, torch_to_np
from estimator import Estimator

# Magic
%matplotlib notebook

## Utilities

In [3]:
def select_hits(hits):
    # Select all barrel hits
    vids = [8, 13, 17]
    barrel_hits = hits[np.logical_or.reduce([hits.volid == v for v in vids])]
    # Re-enumerate the volume and layer numbers for convenience
    volume = pd.Series(-1, index=barrel_hits.index, dtype=np.int8)
    vid_groups = barrel_hits.groupby('volid')
    for i, v in enumerate(vids):
        volume[vid_groups.get_group(v).index] = i
    # This assumes 4 layers per volume (except last volume)
    layer = (barrel_hits.layid / 2 - 1 + volume * 4).astype(np.int8)
    return (barrel_hits[['evtid', 'barcode', 'phi', 'z']]
            .assign(volume=volume, layer=layer))

def select_signal_hits(hits):
    """Select signal hits from tracks that hit all barrel layers"""
    return (hits.groupby(['evtid', 'barcode'])
            # Select tracks that hit every layer at least once
            .filter(lambda x: len(x) >= 10 and x.layer.unique().size == 10)
            # Average duplicate hits together
            .groupby(['evtid', 'barcode', 'layer'], as_index=False).mean())

## Read the data

In [4]:
data_dir = '/global/cscratch1/sd/sfarrell/ACTS/prod_mu10_pt1000_2017_07_29'

In [5]:
n_files = 20

all_files = os.listdir(data_dir)
hits_files = sorted(f for f in all_files if f.startswith('clusters'))
hits_files = [os.path.join(data_dir, f) for f in hits_files[:n_files]]

In [6]:
n_workers = 5
hits = process_files(hits_files, num_workers=n_workers)

Loading /global/cscratch1/sd/sfarrell/ACTS/prod_mu10_pt1000_2017_07_29/clusters_100.csv
Loading /global/cscratch1/sd/sfarrell/ACTS/prod_mu10_pt1000_2017_07_29/clusters_1.csv
Loading /global/cscratch1/sd/sfarrell/ACTS/prod_mu10_pt1000_2017_07_29/clusters_10.csv
Loading /global/cscratch1/sd/sfarrell/ACTS/prod_mu10_pt1000_2017_07_29/clusters_12.csv
Loading /global/cscratch1/sd/sfarrell/ACTS/prod_mu10_pt1000_2017_07_29/clusters_11.csv
Loading /global/cscratch1/sd/sfarrell/ACTS/prod_mu10_pt1000_2017_07_29/clusters_13.csv
Loading /global/cscratch1/sd/sfarrell/ACTS/prod_mu10_pt1000_2017_07_29/clusters_14.csv
Loading /global/cscratch1/sd/sfarrell/ACTS/prod_mu10_pt1000_2017_07_29/clusters_15.csv
Loading /global/cscratch1/sd/sfarrell/ACTS/prod_mu10_pt1000_2017_07_29/clusters_16.csv
Loading /global/cscratch1/sd/sfarrell/ACTS/prod_mu10_pt1000_2017_07_29/clusters_17.csv
Loading /global/cscratch1/sd/sfarrell/ACTS/prod_mu10_pt1000_2017_07_29/clusters_18.csv
Loading /global/cscratch1/sd/sfarrell/ACTS/

In [7]:
print('Hits data shape:', hits.shape)

Hits data shape: (10568614, 7)


In [8]:
%%time

# Select hits
selected_hits = select_hits(hits)
print('selected barrel hits:', selected_hits.shape)
signal_hits = select_signal_hits(selected_hits)
print('signal track hits:', signal_hits.shape)

selected barrel hits: (6655049, 6)
signal track hits: (2736290, 6)
CPU times: user 2min 35s, sys: 1.27 s, total: 2min 36s
Wall time: 2min 36s


In [9]:
%%time

# Gather features into tensor of shape (events, layers, features)
input_data = np.stack(signal_hits.groupby(['evtid', 'barcode'])
                      .apply(lambda x: x[['phi', 'z', 'layer']].values)).astype(np.float32)

# Scale coordinates to approx [-1, 1] for phi and z, [0, 1] for layer number
coord_scale = np.array([np.pi, 1000., 10.])
cov_scale = coord_scale[:2, None] * coord_scale[None, :2]
input_data[:,:] /= coord_scale

CPU times: user 2min 38s, sys: 625 ms, total: 2min 39s
Wall time: 2min 39s


## Define the model

We define an LSTM model in PyTorch which will predict the next hit location.

In [10]:
class Cholesky(torch.autograd.Function):
    """
    Cholesky decomposition with gradient. Taken from
    https://github.com/t-vi/pytorch-tvmisc/blob/master/misc/gaussian_process_regression_basic.ipynb
    """
    @staticmethod
    def forward(ctx, a):
        l = torch.potrf(a, False)
        ctx.save_for_backward(l)
        return l

    @staticmethod
    def backward(ctx, grad_output):
        l, = ctx.saved_variables
        # Gradient is l^{-H} @ ((l^{H} @ grad) * (tril(ones)-1/2*eye)) @ l^{-1}
        # Ideally, this should use some form of solve triangular instead of inverse...
        linv =  l.inverse()
        
        inner = (torch.tril(torch.mm(l.t(), grad_output)) * 
                 torch.tril(1.0 - Variable(l.data.new(l.size(1)).fill_(0.5).diag())))
        s = torch.mm(linv.t(), torch.mm(inner, linv))
        return s

In [11]:
class TrackFilterer(nn.Module):
    """
    A PyTorch module for particle track state estimation and hit prediction.
    
    This module is an RNN which takes a sequence of hits and produces a
    Gaussian shaped prediction for the location of the next hit.
    """
    
    def __init__(self, hidden_dim=5):
        super(TrackFilterer, self).__init__()
        input_dim = 3
        output_dim = 2
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        out_size = input_dim * (input_dim + 3) / 2
        self.fc = nn.Linear(hidden_dim, out_size)
        
    def forward(self, x):
        """Might want to accept also the radius of the target layer."""
        input_size = x.size()
        
        # Initialize the LSTM hidden state
        h = (torch_zeros(input_size[0], self.lstm.hidden_size),
             torch_zeros(input_size[0], self.lstm.hidden_size))
        # Apply the LSTM module
        x, h = self.lstm(x, h)
        # Squash layer axis into batch axis
        x = x.contiguous().view(-1, x.size(-1))
        # Apply linear layer
        output = self.fc(x)
        
        # Extract and transform the gaussian parameters
        means = output[:, :2]
        variances = output[:, 2:4].exp()
        correlations = output[:, 4].tanh()
        
        # Construct the covariance matrix
        covs = torch.bmm(variances[:, :, None], variances[:, None, :]).sqrt()
        covs[:, 0, 1] = covs[:, 0, 1].clone() * correlations
        covs[:, 1, 0] = covs[:, 1, 0].clone() * correlations
        
        # Expand the layer axis again, just for consistency/interpretability
        means = means.contiguous().view(input_size[0], input_size[1], 2)
        covs = covs.contiguous().view(input_size[0], input_size[1], 2, 2)
        return means, covs

In [12]:
def prepare_model_data(data):
    """
    Put a chunk of numpy data into format for the model.
    Reshapes and slices out the input and target features,
    and converts into PyTorch Variable format.    
    """
    # All but last detector layer used as inputs
    inputs = np_to_torch(data[:, :-1])
    # Target includes all but first layer, and drops the layer feature
    targets = np_to_torch(data[:, 1:, :-1])
    return inputs, targets

In [13]:
def gaus_llh_loss(outputs, targets):
    """Custom gaussian log-likelihood loss function"""
    means, covs = outputs
    # Flatten layer axis into batch axis to use batch matrix operations
    means = means.contiguous().view(means.size(0)*means.size(1), means.size(2))
    covs = covs.contiguous().view(covs.size(0)*covs.size(1), covs.size(2), covs.size(3))
    targets = targets.contiguous().view(targets.size(0)*targets.size(1), targets.size(2))
    # Calculate the inverses of the covariance matrices
    inv_covs = torch.stack([cov.inverse() for cov in covs])
    # Calculate the residual error
    # TODO: need to fix for phi discontinuity!!
    res = targets - means
    # Calculate the residual error term
    res_right = torch.bmm(inv_covs, res.unsqueeze(-1)).squeeze(-1)
    res_term = torch.bmm(res[:,None,:], res_right[:,:,None]).squeeze()
    # For the determinant term, we first have to compute the cholesky roots
    diag_chols = torch.stack([Cholesky.apply(cov).diag() for cov in covs])
    log_det = diag_chols.log().sum(1) * 2
    gllh_loss = (res_term + log_det).sum()
    return gllh_loss

## Configure and train the model

In [14]:
# Model config
hidden_dim = 20

# Train config
n_epochs = 50
batch_size = 64
test_frac = 0.1

In [15]:
# Split data into training, validation, and test sets
train_data, test_data = train_test_split(input_data, test_size=test_frac)
# Final preparation of data for the model
train_input, train_target = prepare_model_data(train_data)
test_input, test_target = prepare_model_data(test_data)

In [ ]:
# Construct the model and estimator
estimator = Estimator(
    TrackFilterer(hidden_dim=hidden_dim),
    loss_func=gaus_llh_loss, cuda=cuda)

TrackFilterer (
  (lstm): LSTM(3, 20, batch_first=True)
  (fc): Linear (20 -> 9)
)
Parameters: 2189


In [ ]:
# Train the model
estimator.fit(train_input, train_target,
              valid_input=test_input, valid_target=test_target,
              batch_size=batch_size, n_epochs=n_epochs)

Training samples: 246266
Batches per epoch: 3848
Validation samples: 27363
Epoch 0
  training loss -6.7e+03 time 754.702s
  validate loss -3.66e+06
Epoch 1


In [ ]:
plt.figure()
epochs = np.arange(n_epochs)
p1, = plt.plot(epochs, np.array(estimator.train_losses), c='b', label='Training set')
plt.ylabel('Train loss')
plt.xlabel('Epochs')
plt.twinx()
p2, = plt.plot(epochs, np.array(estimator.valid_losses), c='r', label='Test set')
plt.ylabel('Test loss')
plt.title('Test lost')
plt.legend([p1, p2], ['Training set', 'Test set'])
plt.tight_layout()

## Evaluate model performance

Understanding the loss is a bit difficult. I need to provide some additional metrics.

In [ ]:
test_output = estimator.model(test_input)

### Visualizing predictions

In [ ]:
# Pick out the values to draw
isample = 0
mus = torch_to_np(test_output[0][isample]) * coord_scale[:2]
covs = torch_to_np(test_output[1][isample]) * cov_scale
targets = torch_to_np(test_target[isample]) * coord_scale[:2]

# Draw figure with 9 subplots (3x3) for the predictions on each layer
plt.figure(figsize=(9,8))

for ilay in range(9):
    plt.subplot(3, 3, ilay+1)
    # Construct a gaussian from our prediction
    pred = multivariate_normal(mus[ilay], covs[ilay])
    
    # Compute values on a grid
    wsize = np.array([np.pi/16, 100])
    wlow, whigh = mus[ilay] - wsize, mus[ilay] + wsize
    phi_grid, z_grid = np.mgrid[wlow[0]:whigh[0]:.01, wlow[1]:whigh[1]:1]
    pred_grid = pred.pdf(np.dstack([phi_grid, z_grid]))
    
    # Draw the predicted gaussian
    c = plt.contour(phi_grid, z_grid, pred_grid)
    # Draw the actual hit location
    plt.scatter(targets[ilay, 0], targets[ilay, 1])
    # Draw the mean prediction
    plt.scatter(mus[ilay, 0], mus[ilay, 1], marker='+')

plt.tight_layout()

In [ ]:
layers = np.arange(10)

# Loop over a few samples to draw
for isample in range(4):

    inputs = torch_to_np(test_input[isample]) * coord_scale
    targets = torch_to_np(test_target[isample]) * coord_scale[:2]
    pred_mus = torch_to_np(test_output[0][isample]) * coord_scale[:2]
    pred_covs = torch_to_np(test_output[1][isample]) * cov_scale

    pred_phi = pred_mus[:, 0]
    pred_z = pred_mus[:, 1]
    pred_phi_sig = np.sqrt(pred_covs[:,0,0])
    pred_z_sig = np.sqrt(pred_covs[:,1,1])

    # Draw the means first
    plt.figure(figsize=(9,3))
    plt.subplot(121)
    plt.plot(layers[:-1], inputs[:, 0], 'b.-')
    plt.plot(layers[1:], targets[:,0], 'b.-', label='Data')
    plt.errorbar(layers[1:], pred_phi, yerr=pred_phi_sig, fmt='r.', label='Filter')
    plt.xlabel('layer')
    plt.ylabel('phi [rad]')
    plt.legend(loc=0)

    plt.subplot(122)
    plt.plot(layers[:-1], inputs[:, 1], 'b.-')
    plt.plot(layers[1:], targets[:, 1], 'b.-', label='Data')
    plt.errorbar(layers[1:], pred_z, yerr=pred_z_sig, fmt='r.', label='Filter')
    plt.xlabel('layer')
    plt.ylabel('z [mm]')
    plt.legend(loc=0)

    plt.tight_layout()

### Residual error and pull distributions

Draw the error-weighted mean difference.

In [ ]:
def calc_residuals(preds, targets):
    """
    Calculates the residuals errors for phi and z coordinates.
    Corrects for the delta-phi discontinuity.
    """
    res = preds - targets
    # fix the delta-phi calculation around the discontinuity
    res[res[:,:,0] > np.pi, 0] -= 2*np.pi
    res[res[:,:,0] < -np.pi, 0] += 2*np.pi
    return res

In [ ]:
# Extract the test set data, errors, pulls
preds = torch_to_np(test_output[0]) * coord_scale[:2]
sigmas = np.sqrt(np.diagonal(torch_to_np(test_output[1]), axis1=2, axis2=3)) * coord_scale[:2]
targets = torch_to_np(test_target) * coord_scale[:2]
residuals = calc_residuals(preds, targets)
pulls = residuals / sigmas

In [ ]:
# Fit the pull distributions
phi_pulls = pulls[:, :, 0].flatten()
z_pulls = pulls[:, :, 1].flatten()
fit_x = np.linspace(-5, 5)

phi_pull_mu, phi_pull_std = norm.fit(phi_pulls)
phi_pull_fit = norm.pdf(fit_x, phi_pull_mu, phi_pull_std)

z_pull_mu, z_pull_std = norm.fit(z_pulls)
z_pull_fit = norm.pdf(fit_x, z_pull_mu, z_pull_std)

In [ ]:
# Draw the residuals
plt.figure(figsize=(9,4))

plt.subplot(121)
plt.hist(residuals[:, :, 0].flatten(), bins=50, log=True)
plt.xlabel('$\phi - \phi_{true}$ [rad]')
plt.title('$\phi$ residual error')

plt.subplot(122)
plt.hist(residuals[:, :, 1].flatten(), bins=50, log=True)
plt.xlabel('$z - z_{true}$ [mm]')
plt.title('Z residual error')

plt.tight_layout()

In [ ]:
# Draw the pull distributions
plt.figure(figsize=(9,4))

plt.subplot(121)
plt.hist(phi_pulls, bins=50, range=[-5, 5], normed=True)
fit_label = 'Gaussian fit\n$\mu$ = %.3f\n$\sigma$ = %.3f' % (phi_pull_mu, phi_pull_std)
plt.plot(fit_x, phi_pull_fit, label=fit_label)
plt.xlabel('$(\phi - \phi_{true}) / \sigma_\phi$')
plt.title('$\phi$ pull distribution')
plt.legend(loc=0)

plt.subplot(122)
plt.hist(z_pulls, bins=50, range=[-5, 5], normed=True)
fit_label = 'Gaussian fit\n$\mu$ = %.3f\n$\sigma$ = %.3f' % (z_pull_mu, z_pull_std)
plt.plot(fit_x, z_pull_fit, label=fit_label)
plt.xlabel('$(z - z_{true}) / \sigma_z$')
plt.title('Z pull distribution')
plt.legend(loc=0)

plt.tight_layout()

## Discussion

This model seems to be able to learn trajectories better than the simpler one which only predicts a central value with an MSE loss, which is a rather interesting thing.

It's still not perfect, certainly. There out some outliers in the $\phi$ residual at $\pm \pi$ that I need to investigate. The shapes of the errors and pulls vary a bit as I modify things and rerun, which suggests instability or lack of convergence. Usually the $\phi$ pull distribution is highly assymetric (maybe even multi-modal), but in this current run it is actually fairly well behaved. Usually the z residuals and pull distribution have been very well behaved but in this case we see the pull distribution shows some bias.

The loss during training seems a little unstable, which could be related to the biases observed above. I may be able to improve this.